## AutoGen Studio Agent Workflow API Example

This notebook focuses on demonstrating capabilities of the autogen studio workflow python api.  

- Declarative Specification of an Agent Team
- Loading the specification and running the resulting agent

 

In [1]:
import logging
from autogen_agentchat.logging import ConsoleLogHandler
from autogen_agentchat import EVENT_LOGGER_NAME
logger = logging.getLogger(EVENT_LOGGER_NAME)
logger.handlers = [ConsoleLogHandler()]
logger.setLevel(logging.INFO)


In [2]:
from autogenstudio.provider import Provider 
from autogenstudio.workflowmanager import WorkflowManager
import json 


team_json_spec = json.load(open("team.json")) 
wm = WorkflowManager(team_config=team_json_spec) 

In [3]:
result = await wm.run(task="What is the weather in New York?")


--------------------------------------------------------------------------- 
[2024-10-31T09:54:09.574108]:

What is the weather in New York?
--------------------------------------------------------------------------- 
[2024-10-31T09:54:11.034840], writing_agent:

The weather in New York is currently 73 degrees and sunny.
--------------------------------------------------------------------------- 
[2024-10-31T09:54:11.436336], writing_agent:

TERMINATE
--------------------------------------------------------------------------- 
[2024-10-31T09:54:11.436681], Termination:

Stop message received

In [5]:
result.mes

{'messages': [TextMessage(source='user', content='What is the weather in New York?'),
  ToolCallMessage(source='writing_agent', content=[FunctionCall(id='call_YIA4GdhEi8lcDocytvSDhSz1', arguments='{"city":"New York"}', name='get_weather')]),
  ToolCallResultMessages(source='writing_agent', content=[FunctionExecutionResult(content='The weather in New York is 73 degrees and Sunny.', call_id='call_YIA4GdhEi8lcDocytvSDhSz1')]),
  TextMessage(source='writing_agent', content='The weather in New York is currently 73 degrees and sunny.'),
  StopMessage(source='writing_agent', content='TERMINATE')],
 'usage': '',
 'duration': 1.8642640113830566}